# QIIME2 analysis of 16S V6 sequences of cyanobacterial community on loggerhead sea turtles

qiime2-2022.8 version

### Importing sequences

Trimmed reades genereted by sequencing facility Microsynth can be found in EMBL ENA database under accession number PRJEB68310

#### Imported using Casava 1.8 lanelles format
https://forum.qiime2.org/t/missing-one-or-more-files-when-importing-pe-demultiplexed-files/20530

In [ ]:
!qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path Trimmed_reads \
  --input-format CasavaOneEightLanelessPerSampleDirFmt \
  --output-path demux-V6-trimmed.qza

!qiime demux summarize \
  --i-data demux-V6-trimmed.qza \
  --o-visualization demux-V6-trimmed.qzv

### DADA2 denoising

In [ ]:
!qiime dada2 denoise-paired \
  --i-demultiplexed-seqs demux-V6-trimmed.qza \
  --p-trunc-len-f 258 \
  --p-trunc-len-r 162 \
  --o-table table.qza \
  --o-representative-sequences rep-seqs-V6.qza \
  --o-denoising-stats denoising-stats-V6.qza

!qiime feature-table summarize \
  --i-table table.qza \
  --o-visualization table-V6.qzv \
  --m-sample-metadata-file metadata-V6-v1.tsv
  
!qiime feature-table tabulate-seqs \
  --i-data rep-seqs-V6.qza \
  --o-visualization rep-seqs-V6.qzv

!qiime metadata tabulate \
  --m-input-file denoising-stats-V6.qza \
  --o-visualization denoising-stats-V6.qzv

### Training feature classifiers (SILVA) using CyanoSeq taxonomic annotations

In [ ]:
!qiime tools import \
  --type 'FeatureData[Sequence]' \
  --input-path CyanoSeq/CyanoSeq_1.2.1_SILVA138.1_QIIME2.fasta \
  --output-path CyanoSeq_1.2.1_SILVA138.1_QIIME2_seqs.qza

!qiime tools import \
  --type 'FeatureData[Taxonomy]' \
  --input-format HeaderlessTSVTaxonomyFormat \
  --input-path CyanoSeq/CyanoSeq_1.2.1_SILVA138.1_QIIME2.tsv \
  --output-path CyanoSeq_1.2.1_SILVA138.1_QIIME2_tax.qza

!qiime feature-classifier extract-reads \
  --i-sequences CyanoSeq_1.2.1_SILVA138.1_QIIME2_seqs.qza \
  --p-f-primer GCTAACGCGTTAAGTATCCCGCCTGG \
  --p-r-primer GTCTCTCTAGAGTGCCCAACTTAATG \
  --p-min-length 200 \
  --p-max-length 300 \
  --o-reads ref-seqs-CyanoSeq-V6.qza

!qiime feature-classifier fit-classifier-naive-bayes \
  --i-reference-reads ref-seqs-CyanoSeq-V6.qza \
  --i-reference-taxonomy CyanoSeq_1.2.1_SILVA138.1_QIIME2_tax.qza \
  --o-classifier classifier-CyanoSeq-V6.qza

!qiime feature-classifier classify-sklearn \
  --i-classifier classifier-CyanoSeq-V6.qza \
  --i-reads rep-seqs-V6.qza \
  --o-classification taxonomy-CyanoSeq-V6.qza

!qiime metadata tabulate \
  --m-input-file taxonomy-CyanoSeq-V6.qza \
  --o-visualization taxonomy-CyanoSeq-V6.qzv

### Filtering Cyanobacteria

In [ ]:
!qiime taxa filter-table \
  --i-table table.qza \
  --i-taxonomy taxonomy-CyanoSeq-V6.qza \
  --p-include Cyanobacteriota \
  --p-exclude chloroplast \
  --o-filtered-table table-V6-CyanoSeq-cyanobacteria.qza

!qiime feature-table summarize \
  --i-table table-V6-CyanoSeq-cyanobacteria.qza \
  --o-visualization table-V6-CyanoSeq-cyanobacteria.qzv \
  --m-sample-metadata-file metadata-V6-v2.tsv

!qiime taxa filter-seqs \
  --i-sequences rep-seqs-V6.qza \
  --i-taxonomy taxonomy-CyanoSeq-V6.qza \
  --p-include Cyanobacteriota \
  --p-exclude chloroplast \
  --o-filtered-sequences rep-seqs-V6-CyanoSeq-cyanobacteria.qza

In [3]:
!qiime feature-table summarize \
--i-table table-V6-CyanoSeq-cyanobacteria.qza \
--m-sample-metadata-file metadata-V6-v4.tsv \
--o-visualization table-V6-CyanoSeq-cyanobacteria.qzv

Saved Visualization to: table-V6-CyanoSeq-cyanobacteria.qzv


### Taxa bar plots

In [5]:
!qiime taxa barplot \
 --i-table table-V6-CyanoSeq-cyanobacteria.qza \
 --i-taxonomy taxonomy-CyanoSeq-V6.qza \
 --m-metadata-file metadata-V6-v2.tsv \
 --o-visualization taxa-barplots-V6-cyanovacteria-CyanoSeq.qzv

Saved Visualization to: taxa-barplots-V6-cyanovacteria-CyanoSeq.qzv


### Phylogenetic tree construction

In [ ]:
!qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences rep-seqs-V6-CyanoSeq-cyanobacteria.qza \
  --o-alignment aligned-rep-seqs-V6-CyanoSeq-cyanobacteria.qza \
  --o-masked-alignment masked-aligned-rep-seqs-V6-CyanoSeq-cyanobacteria.qza \
  --o-tree unrooted-tree-V6-CyanoSeq-cyanobacteria.qza \
  --o-rooted-tree rooted-tree-V6-CyanoSeq-cyanobacteria.qza

### Alpha and beta core metrics

In [2]:
!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny rooted-tree-V6-CyanoSeq-cyanobacteria.qza \
  --i-table table-V6-CyanoSeq-cyanobacteria.qza \
  --p-sampling-depth 10098 \
  --m-metadata-file metadata-V6-v2.tsv \
  --output-dir core-metrics-results-10098

Saved FeatureTable[Frequency] to: core-metrics-results-10098/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results-10098/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results-10098/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results-10098/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results-10098/evenness_vector.qza
Saved DistanceMatrix to: core-metrics-results-10098/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results-10098/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results-10098/jaccard_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results-10098/bray_curtis_distance_matrix.qza
Saved PCoAResults to: core-metrics-results-10098/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results-10098/weighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results-10098/jaccard_pcoa_re

Permanova with Bray-Curtis distance

In [3]:
!qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results-10098/bray_curtis_distance_matrix.qza \
  --m-metadata-file metadata-V6-v2.tsv \
  --m-metadata-column Age \
  --o-visualization core-metrics-results-10098/permanova_bc_Age.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results-10098/permanova_bc_Age.qzv


Permanova with Jaccard distance

In [1]:
!qiime diversity beta-group-significance \
  --i-distance-matrix jaccard_distance_matrix.qza \
  --m-metadata-file metadata-V6-v2.tsv \
  --m-metadata-column Age \
  --o-visualization permanova_jaccard_Age.qzv \
  --p-pairwise

Saved Visualization to: permanova_jaccard_Age.qzv


Permanova with unweighted UniFrac

In [3]:
!qiime diversity beta-group-significance \
  --i-distance-matrix unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file metadata-V6-v2.tsv \
  --m-metadata-column Age \
  --o-visualization permanova_uw_unifrac_Age.qzv \
  --p-pairwise

Saved Visualization to: permanova_uw_unifrac_Age.qzv


Permanova with weighted UniFrac

In [2]:
!qiime diversity beta-group-significance \
  --i-distance-matrix weighted_unifrac_distance_matrix.qza \
  --m-metadata-file metadata-V6-v2.tsv \
  --m-metadata-column Age \
  --o-visualization permanova_w_unifrac_Age.qzv \
  --p-pairwise

Saved Visualization to: permanova_w_unifrac_Age.qzv


Aitchinson distance

In [4]:
!qiime dev refresh-cache

!qiime deicode rpca \
    --i-table table-V6-CyanoSeq-cyanobacteria.qza \
    --p-min-feature-count 10 \
    --p-min-sample-count 500 \
    --o-biplot rPCA-ordination.qza \
    --o-distance-matrix aitchison-distance.qza

!qiime emperor biplot \
    --i-biplot rPCA-ordination.qza \
    --m-sample-metadata-file metadata-V6-v2.tsv \
    --m-feature-metadata-file taxonomy-CyanoSeq-V6.qza \
    --o-visualization rPCA-biplot.qzv \
    --p-number-of-features 5

QIIME is caching your current deployment for improved performance. This may take a few moments and should only happen once per deployment.
Saved PCoAResults % Properties('biplot') to: rPCA-ordination.qza
Saved DistanceMatrix to: aitchison-distance.qza
Saved Visualization to: rPCA-biplot.qzv


Permanova with Aitchinson distance

In [5]:
!qiime diversity beta-group-significance \
  --i-distance-matrix aitchison-distance.qza \
  --m-metadata-file metadata-V6-v2.tsv \
  --m-metadata-column Age \
  --o-visualization permanova_aitchison_Age.qzv \
  --p-pairwise

Saved Visualization to: permanova_aitchison_Age.qzv


### PERMANOVA - coast comparisson

In [1]:
!qiime diversity beta-group-significance \
  --i-distance-matrix bray_curtis_distance_matrix.qza \
  --m-metadata-file metadata-V6-v4.tsv \
  --m-metadata-column coast \
  --o-visualization permanova_bc_coast.qzv \
  --p-pairwise

!qiime diversity beta-group-significance \
  --i-distance-matrix jaccard_distance_matrix.qza \
  --m-metadata-file metadata-V6-v4.tsv \
  --m-metadata-column coast \
  --o-visualization permanova_jaccard_coast.qzv \
  --p-pairwise

!qiime diversity beta-group-significance \
  --i-distance-matrix unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file metadata-V6-v4.tsv \
  --m-metadata-column coast \
  --o-visualization permanova_uw_unifrac_coast.qzv \
  --p-pairwise

!qiime diversity beta-group-significance \
  --i-distance-matrix weighted_unifrac_distance_matrix.qza \
  --m-metadata-file metadata-V6-v4.tsv \
  --m-metadata-column coast \
  --o-visualization permanova_w_unifrac_coast.qzv \
  --p-pairwise

!qiime diversity beta-group-significance \
  --i-distance-matrix aitchison-distance.qza \
  --m-metadata-file metadata-V6-v4.tsv \
  --m-metadata-column coast \
  --o-visualization permanova_aitchison_coast.qzv \
  --p-pairwise

Saved Visualization to: permanova_bc_coast.qzv
Saved Visualization to: permanova_jaccard_coast.qzv
Saved Visualization to: permanova_uw_unifrac_coast.qzv
Saved Visualization to: permanova_w_unifrac_coast.qzv
Saved Visualization to: permanova_aitchison_coast.qzv


### Core features

In [6]:
!qiime feature-table core-features \
    --i-table table-V6-CyanoSeq-cyanobacteria.qza \
    --o-visualization core-features.qzv

Saved Visualization to: core-features.qzv


Juveniles

In [7]:
!qiime feature-table filter-samples \
  --i-table table-V6-CyanoSeq-cyanobacteria.qza \
  --m-metadata-file metadata-V6-v2.tsv \
  --p-where "[Age]='juvenile'" \
  --o-filtered-table table-juvenile.qza

!qiime feature-table core-features \
    --i-table table-juvenile.qza \
    --o-visualization core-features-juvenile.qzv

Saved FeatureTable[Frequency] to: table-juvenile.qza
Saved Visualization to: core-features-juvenile.qzv


Subadults

In [8]:
!qiime feature-table filter-samples \
  --i-table table-V6-CyanoSeq-cyanobacteria.qza \
  --m-metadata-file metadata-V6-v2.tsv \
  --p-where "[Age]='sub-adult'" \
  --o-filtered-table table-sub-adult.qza

!qiime feature-table core-features \
    --i-table table-sub-adult.qza \
    --o-visualization core-features-sub-adult.qzv

Saved FeatureTable[Frequency] to: table-sub-adult.qza
Saved Visualization to: core-features-sub-adult.qzv


Adult

In [9]:
!qiime feature-table filter-samples \
  --i-table table-V6-CyanoSeq-cyanobacteria.qza \
  --m-metadata-file metadata-V6-v2.tsv \
  --p-where "[Age]='adult'" \
  --o-filtered-table table-adult.qza

!qiime feature-table core-features \
    --i-table table-adult.qza \
    --o-visualization core-features-adult.qzv

Saved FeatureTable[Frequency] to: table-adult.qza
Saved Visualization to: core-features-adult.qzv
